In [3]:
import os
import pdb
import time
import math
import shutil
import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import tools_data as td
import dataLaoder as dl
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import itertools

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from torch.utils.tensorboard import SummaryWriter

from tqdm.notebook import tqdm_notebook

from torch.utils.data import  DataLoader

In [4]:
class State(object):
    def __init__(self,model,optim) :
        self.model = model
        self.optim = optim
        self.epoch ,self.iter =0,0

    def getEpoch(self):
        return self.epoch

In [5]:
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions

VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ],
    "VGG19": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        512,
        "M",
    ],
}


class VGG_net(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(VGG_net, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_types["VGG11"])

        self.fcs = nn.Sequential(
            nn.Linear(512 , 512),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]
                in_channels = x
            elif x == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)


"""if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = VGG_net(in_channels=3, num_classes=1000).to(device)
    print(model)
    ## N = 3 (Mini batch size)
    x = torch.randn(1, 3, 32, 32).to(device)
    print(model(x).shape)"""

'if __name__ == "__main__":\n    device = "cuda" if torch.cuda.is_available() else "cpu"\n    model = VGG_net(in_channels=3, num_classes=1000).to(device)\n    print(model)\n    ## N = 3 (Mini batch size)\n    x = torch.randn(1, 3, 32, 32).to(device)\n    print(model(x).shape)'

In [6]:
svhn_train_x,svhn_train_y = td.load_svhn_train('./data/svhn/train_32x32.mat')
svhn_test_x,svhn_test_y = td.load_svhn_train('./data/svhn/test_32x32.mat')

svhn_train_x = svhn_train_x.transpose([0, 3, 2, 1])

In [7]:
cifar10_train_x,cifar_train_y = td.load_cifar10_set('./data/cifar-10/cifar-10-batches-py/data_batch_')
cifar10_test_x,cifar_test_y = td.load_cifar10_one('./data/cifar-10/cifar-10-batches-py/test_batch')

cifar10_train_x = cifar10_train_x.transpose([0, 3, 2, 1])
cifar10_test_x = cifar10_test_x.transpose([0, 3, 2, 1])

In [8]:
"""in_train = dl.MyDataSet(cifar10_train_x,cifar_train_y)
in_test = dl.MyDataSet(cifar10_test_x,np.array(cifar_test_y))


out_train = dl.MyDataSet(svhn_train_x,svhn_train_y)
out_test = dl.MyDataSet(svhn_test_x,svhn_test_y)"""


'in_train = dl.MyDataSet(cifar10_train_x,cifar_train_y)\nin_test = dl.MyDataSet(cifar10_test_x,np.array(cifar_test_y))\n\n\nout_train = dl.MyDataSet(svhn_train_x,svhn_train_y)\nout_test = dl.MyDataSet(svhn_test_x,svhn_test_y)'

In [9]:
def epoch_batches(data_x,data_y,nb_batches,nb_cls):
    
    indx_cls,indx_no_cls = [],[]
    
    left_data = []
    right_data = []
    y_b = []
    y = []
    one_batch_x = []
    one_batch_y = []
    x = []
    
    
    for m in range(len(np.unique(data_y))):
        indx_cls.append(np.where(data_y == m)[0])
        indx_no_cls.append(np.where(data_y != m)[0])
    
    for btc in range(nb_batches):
        for cl in range(0,nb_cls):
            
            random.shuffle(indx_cls[cl])
            random.shuffle(indx_no_cls[cl])
            
            
            left_data.append(data_x[indx_cls[cl][0:10]][:])
            right_data.append(data_x[indx_cls[cl][10:20]][:])
            y_b.append([1 for i in range(0,10)])
            
            cla = random.shuffle(indx_cls[cl])
            
            left_data.append(data_x[indx_cls[cl][0:30]][:])
            right_data.append(data_x[indx_no_cls[cl][30:60]][:])
            y_b.append([0 for i in range(0,30)])
        
        left_data = list(itertools.chain(*left_data))
        right_data = list(itertools.chain(*right_data))
        one_batch_x = left_data+right_data
        one_batch_y = list(itertools.chain(*y_b)) + list(itertools.chain(*y_b))
        
        x.append(one_batch_x)
        y.append(one_batch_y)
        
        one_batch_x = []
        one_batch_y = []
        left_data = []
        right_data =[]
        y_b = []
        
    x = list(itertools.chain(*x))
    y = list(itertools.chain(*y))
    
    
    data = dl.MyDataSet(np.array(x),np.array(y))
    data = DataLoader(dataset=data,batch_size=800 ,shuffle = False , num_workers=2)
            
    return data
            
            
            
            
        
    

In [10]:
#data = epoch_batches(cifar10_train_x,cifar_train_y,75,10) 

In [11]:
"""
tmp = 0         
#d_train = epoch_batches(cifar10_train_x,cifar_train_y,75,10) 
    
for x,y in data :
            
            
    #optim.zero_grad()
            
    #xhat = net(x)
            

    anchors_x = x[0:x.shape[0]//2]
    y = y[0:y.shape[0]//2]
            
            
    pos_neg_x = x[x.shape[0]//2:x.shape[0]]
    # pos_neg_y = y[xhat.shape[0]//2:xhat.shape[0]]
        
    if tmp == 0 : 
        #plt.figure(figsize=(40,900))
        print(y.shape)
        #t = 1
        
        #for i in range(300):
            
            # print(y[i])
            #plt.subplot(300,2,t)

            #plt.imshow(np.array(anchors_x[i]).transpose([ 2, 1, 0]))
            #t += 1 
            #plt.subplot(300,2,t)
            #t += 1 
            #plt.imshow(np.array(pos_neg_x[i]).transpose([ 2, 1, 0]))
        #plt.show()
    tmp += 1
        """
    

'\ntmp = 0         \n#d_train = epoch_batches(cifar10_train_x,cifar_train_y,75,10) \n    \nfor x,y in data :\n            \n            \n    #optim.zero_grad()\n            \n    #xhat = net(x)\n            \n\n    anchors_x = x[0:x.shape[0]//2]\n    y = y[0:y.shape[0]//2]\n            \n            \n    pos_neg_x = x[x.shape[0]//2:x.shape[0]]\n    # pos_neg_y = y[xhat.shape[0]//2:xhat.shape[0]]\n        \n    if tmp == 0 : \n        #plt.figure(figsize=(40,900))\n        print(y.shape)\n        #t = 1\n        \n        #for i in range(300):\n            \n            # print(y[i])\n            #plt.subplot(300,2,t)\n\n            #plt.imshow(np.array(anchors_x[i]).transpose([ 2, 1, 0]))\n            #t += 1 \n            #plt.subplot(300,2,t)\n            #t += 1 \n            #plt.imshow(np.array(pos_neg_x[i]).transpose([ 2, 1, 0]))\n        #plt.show()\n    tmp += 1\n        '

In [12]:
#in_train_laoder = DataLoader(dataset=in_train,batch_size=800 ,shuffle = True,num_workers=2)
#in_test_laoder = dataTestLoader = DataLoader(dataset=in_test,batch_size=200,shuffle = True,num_workers=2)

#out_train_laoder = dataTestLoader = DataLoader(dataset=out_train,batch_size=150,shuffle = True,num_workers=2)
#out_test_laoder = dataTestLoader = DataLoader(dataset=out_test,batch_size=1,shuffle = True,num_workers=2)

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
def euclidean_dist(x, y):
    m, n = x.size(0), y.size(0)
    xx = torch.pow(x, 2).sum(1, keepdim=True).expand(m, n)
    yy = torch.pow(y, 2).sum(1, keepdim=True).expand(n, m).t()
    dist = xx + yy
    dist.addmm_(1, -2, x, y.t())
    dist = dist.clamp(min=1e-12).sqrt()  # for numerical stability
    return dist

In [15]:
class MetricLearning(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(MetricLearning, self).__init__()
 
    def forward(self, y_compare, pos_neg_x,anchors_x, smooth=1):        
        
    
        Dw = euclidean_dist(anchors_x , pos_neg_x)
    
        p_p = 0.5*y_compare*torch.pow(Dw,2)
        
        
        m = torch.maximum((10.0-Dw),torch.tensor(0.0))
        
        p_n = 0.5*(1-y_compare)*torch.pow(m,2)
        
        
        # print(torch.mean(p_p),torch.mean(p_n))
        return torch.mean(p_p+ p_n)

In [16]:
    net = VGG_net(in_channels=3, num_classes=1000).to(device)
    optim = torch.optim.Adam(net.parameters(),lr=0.0001,    weight_decay = 1e-4)

In [17]:
from pathlib import Path
savepath = Path('models.pch')

In [18]:
def train_net(x_train,y_train,data_out):

    #writer = SummaryWriter("runs")
    
    
    
    if savepath.is_file():
        print('Restarting from previous state')
        with savepath.open('rb') as fp :
            state = torch.load(fp)
            myloss = MetricLearning()
            
    else : 
        net = VGG_net(in_channels=3, num_classes=1000).to(device)
        optim = torch.optim.Adam(net.parameters(),lr=0.0001,    weight_decay = 1e-4)
        myloss = MetricLearning()
        
        state = State(net, optim)
        
    l = []
    
    state.optim = torch.optim.Adam(state.model.parameters(),lr=0.00001 ,    weight_decay = 1e-4)
    #state = State(net, optim)
        
    
    for epoch in tqdm_notebook(range(state.epoch,3000), desc = 'Epochs'):
        
        d_train = epoch_batches(cifar10_train_x,cifar_train_y,75,10) 
        
        tmp = 0
        
        if epoch == 150 :
            state.optim = torch.optim.Adam(net.parameters(),lr=0.00001 ,    weight_decay = 1e-4) 
            
        if epoch == 2250 :
            state.optim =  torch.optim.Adam(net.parameters(),lr=0.00001 ,    weight_decay = 1e-4) 
        
        for x,y in d_train :
            
            x = x.to(device)
            y = y.to(device)
            
            state.optim.zero_grad()
            
            
            xhat = state.model(x)
            

            anchors_x = xhat[0:xhat.shape[0]//2]
            y_compare = y[0:y.shape[0]//2]
            
            
            pos_neg_x = xhat[xhat.shape[0]//2:xhat.shape[0]]
            #pos_neg_y = y[xhat.shape[0]//2:xhat.shape[0]]
            
            #y_compare = torch.eq( anchors_y , pos_neg_y).int()
            #y_compare = y_compare.to(device)
            
            #Dw = (anchors_x - pos_neg_x).pow(2).sum(1).sqrt()

            
            #loss = torch.mean((0.5*torch.pow((y_compare*Dw),2) + 0.5*torch.pow(torch.multiply((1-y_compare),torch.max((10.0-Dw))),2)**2))
            
            
            
            loss = myloss(y_compare, pos_neg_x,anchors_x, smooth=1)
            
            tmp += loss
            loss.backward()
            state.optim.step()
            state.iter += 1
        
        with savepath.open('wb') as fp :
            state.epoch = epoch+1
            torch.save(state,fp)
            
        print(tmp/75)
            
        l.append(tmp/75)
        

        
    return net , l
            
            
              

In [19]:
res = train_net([],[],[])

Restarting from previous state


Epochs:   0%|          | 0/2654 [00:00<?, ?it/s]

/home/hider/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1050.)
  


tensor(10.2074, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2067, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2176, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2122, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2097, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2118, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2178, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2105, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2168, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2170, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2107, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2190, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2014, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2219, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2126, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2109, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2084, device='cuda:0', grad_fn=<DivBackward0>)
tensor(10.2067, device='cuda:0'

KeyboardInterrupt: 

In [ ]:
net,loss = res

In [ ]:
    myloss = MetricLearning()    
    for x,y in in_train_laoder : 
            x = x.to(device)
            xhat = net(x)
            anchors_x = xhat[0:xhat.shape[0]//2]
            anchors_y = y[0:y.shape[0]//2]
            
            
            pos_neg_x = xhat[xhat.shape[0]//2:xhat.shape[0]]
            pos_neg_y = y[xhat.shape[0]//2:xhat.shape[0]]
            
            y_compare = torch.eq( anchors_y , pos_neg_y).int()
            y_compare = y_compare.to(device)
            lt = myloss(y_compare, pos_neg_x,anchors_x, smooth=1)
            
    print(lt)